In [1]:
import numpy as np
import matplotlib.pyplot as plt
import timewarp_lib.load_model as lm
import timewarp_lib.train_utils as tu
import torch
import pandas as pd
import json
import numpy as np
import matplotlib

/home/travers/miniconda3/envs/timewarpvae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cmap = plt.get_cmap("viridis")

In [3]:

####clean_model_dirs = ["../results/overnight/20230921-093159.705744/savedmodel"] # TenDTWModel



In [4]:
import matplotlib
font = {        'size'   : 22}

matplotlib.rc('font', **font)

In [5]:
datatype = "train"
dtype=torch.float
device="cpu"
batch_size=100
training_data_timing_noise=0
TRAJLEN=200

dataName = "../data/trainTest2DLetterARescaled.npz"
# the datafile has information on how the data was cleaned
# the learned/applied model is on clean data
# so we need to convert back to trajectory data
rawdata = loaded_data_dict = np.load(dataName)

In [6]:
def load_this_model(modelname,train):
    model = lm.LoadedModel(modelname)
    ts = torch.tensor(np.linspace(0,1,train.shape[1]).reshape((1,train.shape[1],1)), dtype=torch.float).expand((train.shape[0],train.shape[1],1))
    recons, mu, logvar, scaled_ts = model.model.noiseless_forward(train,ts)

    return mu, recons, scaled_ts, model

In [21]:
from IPython.display import display

def plot_1d_sweep(modelname, sweepaxis=0,num_trajs_to_plot=101):
    cmap = plt.get_cmap("viridis")
    model = lm.LoadedModel(modelname)
    print(model.modeldata)
    # Create a new plot
    model_data_path = str(model.modeldata["datafile"])
    modeldata_info = np.load("../"+ model_data_path)
    pose_scaling = modeldata_info["pose_scaling"]
    pose_mean = modeldata_info["pose_mean"]
    train = torch.tensor(rawdata["train"],dtype=torch.float)
    embed, recons, scaled_ts, model = load_this_model(modelname,train)
    #print(embed)
    latent_dim = model.modeldata["latent_dim"]
    num_timesteps = TRAJLEN
    
    scaled_ts = scaled_ts.detach().cpu().numpy()
    npts = np.mean(scaled_ts,axis=0,keepdims=True)
    scaled_ts = torch.tensor(npts,dtype=torch.float).expand((num_trajs_to_plot,num_timesteps,1))
    
    embednp = embed.detach().cpu().numpy()
    sweep_embed = np.zeros(shape=(num_trajs_to_plot,latent_dim))    
    for i in range(latent_dim):
        sweep_embed[:,i] = np.median(embednp[:,i])
    med_sweep_embed = torch.tensor(sweep_embed,dtype=torch.float)
    med_traj = model.model.decoder.decode(med_sweep_embed[:1],scaled_ts[:1]).detach().numpy()

    unscale_recons = med_traj
    
    #xs = unscale_recons[0,:TRAJLEN,0]
    #ys = unscale_recons[0,:TRAJLEN,1]
    #zs = unscale_recons[0,:TRAJLEN,2]
    #axes.plot(xs,ys,zs,color="k",linewidth=10)
    #
    median_result_trajs = model.model.decoder.decode(med_sweep_embed,scaled_ts).detach().numpy()
    for i in range(latent_dim):
        sweep_embed[:,i] = np.median(embednp[:,i])
    sweep_embed[:,sweepaxis] = np.percentile(embednp[:,sweepaxis],np.linspace(2,98,num_trajs_to_plot).astype(int))
    
    sweep_embed = torch.tensor(sweep_embed,dtype=torch.float)
    
    result_trajs = model.model.decoder.decode(sweep_embed,scaled_ts).detach().numpy()

    
    sweep_embed = sweep_embed.detach().cpu().numpy()
    recons = result_trajs#.detach().cpu().numpy()
    train = train.detach().cpu().numpy()
    
    
    # matplotlib doesn't do raytracing, so we gotta manually plot in order
    
    minembed = np.min(sweep_embed[:,sweepaxis]) if sweep_embed.shape[1]>0 else 0
    maxembed = np.max(sweep_embed[:,sweepaxis]) if sweep_embed.shape[1]>0 else 0
    order = np.argsort(-sweep_embed[:,sweepaxis]) if model.modeldata["latent_dim"] > 0 else np.arange(len(train))
    for rto, raytracing_order in enumerate([order]):#,np.argsort(embed.flatten())]):
        for name,data in [("Reconstructed",result_trajs)]: #
            unscale_recons = data

            for framenumber, trajid in enumerate(raytracing_order):
                figure = plt.figure(figsize=(10,10))
                axes = plt.axes()
                xs = unscale_recons[trajid,:TRAJLEN,0]
                ys = unscale_recons[trajid,:TRAJLEN,1]
                #colorval = cmap((sweep_embed[trajid,sweepaxis]-minembed)/(maxembed-minembed)) if embed.shape[1] > 0  else "black"
                
                #colorval = (colorval[0],colorval[1],colorval[2],0.8)
                # backfill in white first
                #axes.plot(xs,ys,color=(1,1,1),linewidth=7)
                axes.plot(xs,ys,color="black",linewidth=7)

                plt.xlim(-3,-3+6)
                plt.ylim(-3,-3+6)
                plt.axis("off")
                figure.savefig(f"animatedLatentLetterA/latent{sweepaxis}-frame{framenumber}.png", bbox_inches='tight', pad_inches=0.1)
                #plt.show()
                plt.close()

In [23]:
for latdimplot in range(16):
    plot_1d_sweep("../results/rescaled/20231011-003229.166126/savedmodel",latdimplot)

{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011-003229.166126/log', dtype='<U43'), 'batch_size': array(64), 'log_to_wandb_name': array('rescaled', dtype='<U8'), 'scalar_timewarper_name': array('modeled_scalar_timewarper', dtype='<U25'), 'scaltw_granularity': array(50), 'scaltw_emb_conv_layers_channels': array([16, 32, 32, 64, 64, 64]), 'scaltw_emb_conv_layers_strides': array([1, 2, 1, 2, 1, 2]), 'scaltw_emb_conv_layers_kernel_sizes': array([3, 3, 3, 3, 3, 3]), 'scaltw_emb_fc_layers_num_features': array([], dtype=float64), 'encoder_name': array('convol

after fc:  torch.Size([25000, 500])
after fc:  torch.Size([25000, 64])
motion_model_input torch.Size([200, 1])
after fc:  torch.Size([200, 500])
after fc:  torch.Size([200, 500])
after fc:  torch.Size([200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011

after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011-003229.166126/log', dtype='<U43'), 'batch_size': array(64), 'log_to_wandb_name': array('rescaled', dtype='<U8'), 'scalar_timewarper_name': array('modeled_scalar_timewarper', dtype='<U25'), 'scaltw_granularity': array(50), 'scaltw_emb_conv_layers_channels': array([16, 32, 32, 64, 64, 64]),

after fc:  torch.Size([25000, 500])
after fc:  torch.Size([25000, 64])
motion_model_input torch.Size([200, 1])
after fc:  torch.Size([200, 500])
after fc:  torch.Size([200, 500])
after fc:  torch.Size([200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011

after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011-003229.166126/log', dtype='<U43'), 'batch_size': array(64), 'log_to_wandb_name': array('rescaled', dtype='<U8'), 'scalar_timewarper_name': array('modeled_scalar_timewarper', dtype='<U25'), 'scaltw_granularity': array(50), 'scaltw_emb_conv_layers_channels': array([16, 32, 32, 64, 64, 64]),

after fc:  torch.Size([25000, 500])
after fc:  torch.Size([25000, 64])
motion_model_input torch.Size([200, 1])
after fc:  torch.Size([200, 500])
after fc:  torch.Size([200, 500])
after fc:  torch.Size([200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011

after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
motion_model_input torch.Size([20200, 1])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 500])
after fc:  torch.Size([20200, 64])
{'dtype_string': array('float', dtype='<U5'), 'datafile': array('data/trainTest2DLetterARescaled.npz', dtype='<U35'), 'model_save_dir': array('results/rescaled/20231011-003229.166126/savedmodel', dtype='<U50'), 'num_epochs': array(20000), 'latent_dim': array(16), 'device': array('cuda', dtype='<U4'), 'dtype': torch.float32, 'traj_len': array(200), 'traj_channels': array(2), 'beta': array(0.001), 'training_data_added_timing_noise': array(0.1), 'logname': array('results/rescaled/20231011-003229.166126/log', dtype='<U43'), 'batch_size': array(64), 'log_to_wandb_name': array('rescaled', dtype='<U8'), 'scalar_timewarper_name': array('modeled_scalar_timewarper', dtype='<U25'), 'scaltw_granularity': array(50), 'scaltw_emb_conv_layers_channels': array([16, 32, 32, 64, 64, 64]),